In [1]:
# import packages
import numpy as np
import pandas as pd
from keras.datasets import mnist
from scipy import stats
from scipy.spatial.distance import cdist
from time import time

# load MNIST dataset
(train_X, train_Y), (test_X, test_Y) = mnist.load_data()

In [2]:
# subset data by taking 10% of data from every label group 
sub_indices =  []
for label in range(10):
  curr_indices = np.argwhere(train_Y==label)  # get indices of current label
  np.random.shuffle(curr_indices)  # shuffle indices
  curr_indices = curr_indices[:1000]  # take 1000 of shuffled indices
  sub_indices += curr_indices.flatten().astype(int).tolist() # add to list of all indices to include in the subset

sub_X, sub_Y = train_X[sub_indices], train_Y[sub_indices]

# reshape from 28x28 to flattened representation
sub_X = sub_X.reshape(10000, 784)

test_X_shaped = test_X.reshape(10000, 784)

In [3]:
# compute various distances from each point to every other point
train_cosine = cdist(sub_X, sub_X, 'cosine')
train_euclidean = cdist(sub_X, sub_X, 'euclidean')
train_manhattan = cdist(sub_X, sub_X, 'cityblock')

test_cosine = cdist(test_X_shaped, sub_X, 'cosine')
test_euclidean = cdist(test_X_shaped, sub_X, 'euclidean')
test_manhattan = cdist(test_X_shaped, sub_X, 'cityblock')

In [4]:
def knn(distance_matrix, train_labels, test_labels, k):
  '''
  Implements K nearest neighbors algorithm to predict image labels

  Args:
      distance_matrix: matrix of distances between train and test points
      train_labels: labels for training data
      test_labels: labels for testing data
      k: number of neighbors to use for class prediction

  Returns:
      Class predictions for all points in the test dataset
  '''
  n = test_labels.shape[0]  # number of datapoints in test set

  index_array = np.argpartition(distance_matrix, kth=k, axis=1)[:, :k]  # take the nearest k neighbors
  predictions = stats.mode(train_labels[index_array], axis=1)[0].flatten()  # assign class prediction to label that occurs most often
  accuracy = np.equal(predictions, test_labels).astype(int).sum()/n  # compute accuracy of predictions

  return accuracy

In [5]:
k = 3

# for each distance measure, predict labels using KNN on both training and testing datasets and print results
train_cosine_acc = knn(train_cosine, sub_Y, sub_Y, k)
test_cosine_acc = knn(test_cosine, sub_Y, test_Y, k)
print("Training Cosine: ", train_cosine_acc)
print("Testing Cosine: ", test_cosine_acc, "\n")

train_euclidean_acc = knn(train_euclidean, sub_Y, sub_Y, k)
test_euclidean_acc = knn(test_euclidean, sub_Y, test_Y, k)
print("Training Euclidean: ", train_euclidean_acc)
print("Testing Euclidean: ", test_euclidean_acc, "\n")

train_manhattan_acc = knn(train_manhattan, sub_Y, sub_Y, k)
test_manhattan_acc = knn(test_manhattan, sub_Y, test_Y, k)
print("Training Manhattan: ", train_manhattan_acc)
print("Testing Manhattan: ", test_manhattan_acc)

Training Cosine:  0.9756
Testing Cosine:  0.9564 

Training Euclidean:  0.9721
Testing Euclidean:  0.9474 

Training Manhattan:  0.9666
Testing Manhattan:  0.9385
